In [2]:
import json
import logging
import os
from  nltk import ngrams
import multiprocessing as mp
import matplotlib.pyplot
import data

In [44]:
DEFAULT_MALE_NOUNS = [
    'countryman',
    'fraternal','wizards','manservant','fathers','divo','actor','bachelor','papa',
    'dukes','barman','countrymen','brideprice','hosts','potential_suitors','airmen','andropause',
    'penis','prince','governors','abbot','men','widower','gentlemen','sorcerers','sir',
    'bridegrooms','baron','househusbands','gods','nephew','widowers','lord','brother',
    'grooms','priest','adultors','andrology','bellboys','his','marquis','princes','emperors',
    'stallion','chairman','monastery','priests','boyhood','fellas','king','dudes',
    'daddies','manservant','semen','spokesman','tailor','cowboys','dude','bachelors','barbershop','emperor','daddy',
    'masculism','guys','enchanter','guy','fatherhood','androgen','cameramen','godfather',
    'strongman','god','patriarch','uncle','chairmen','sir','brotherhood','host','testosterone',
    'husband','dad','steward','males','cialis','spokesmen','pa','beau','stud','bachelor',
    'wizard','sir','nephews','fathered','bull','beaus','councilmen','landladies','grandson',
    'fiances','stepfathers','horsewomen','grandfathers','adultor','schoolboy','rooster','grandsons',
    'bachelor','cameraman','dads','him','master','lad','policeman','monk','actors','salesmen',
    'boyfriend','councilman','fella','statesman','paternal','chap','landlord','brethren','lords',
    'blokes','fraternity','bellboy','duke','ballet_dancer','dudes','fiance','colts',
    'husbands','suitor','maternity','he','businessman','masseurs',
    'hero','deer','busboys','boyfriends','kings','brothers','masters','stepfather','brides',
    'son','studs','cowboy','mentleman','sons','baritone','salesman','paramour','male_host',
    'monks','menservants',"mr.",'headmasters','lads','congressman','airman','househusband',
    'priest','barmen','barons','abbots','handyman','beard','fraternities','stewards','colt',
    'czar','stepsons','himself','boys','lions','gentleman','his','masseur','bulls','uncles','bloke','beards',
    'hubby','lion','sorcerer','macho','father','gays','male','waiters','stepson','prostatic_utricle',
    'businessmen','heir','waiter','headmaster','man','governor','god','bridegroom','grandpa',
    'groom','dude','gay','gents','boy','grandfather','gelding','paternity',
    'roosters','prostatic_utricle','priests','manservants','stailor','busboy','heros'
    ]

#yusu added special
DEFAULT_MALE_SPECIAL = [
    'sperm','beard','mustache',
    'ejaculation','erection','scrotum','penis','testicles','epididymis'
]

#yusu added words from the rule file
DEFAULT_FEMALE_NOUNS = [
    'woman', 'women', 'ladies', 'female', 'females', 'girl', 'girlfriend',
    'girlfriends', 'girls', 'her', 'hers', 'lady', 'she', 'wife', 'wives','countrywoman',
'sororal','witches','maidservant','mothers','diva','actress','spinster','mama',
    'duchesses','barwoman','countrywomen','dowry','hostesses','airwomen','princess',
    'governesses','abbess','women','widow','ladies','sorceresses','madam','brides',
    'baroness','housewives','godesses','niece','widows','lady',
'sister','brides','nun','adultresses','obstetrics','bellgirls','her','marchioness',
    'princesses','empresses','mare','chairwoman','convent','priestesses','girlhood',
    'ladies','queen','gals','mommies','maid','female_ejaculation','spokeswoman','seamstress',
    'cowgirls','chick','spinsters','empress','mommy','feminism','gals','enchantress','gal','motherhood','estrogen'
    'camerawomen','godmother','strongwoman','goddess','matriarch','aunt','chairwomen',"ma'am",
    'sisterhood','hostess','estradiol','wife','mom','stewardess','females','viagra',
    'spokeswomen','ma','belle','minx','maiden','witch','miss','nieces','mothered',
    'cow','belles','councilwomen','landlords','granddaughter','fiancees','stepmothers',
    'horsemen','grandmothers','adultress','schoolgirl','hen','granddaughters','bachelorette',
    'camerawoman','moms','her','mistress','lass','policewoman','nun','actresses',
    'saleswomen','girlfriend','councilwoman','lady','stateswoman','maternal',
    'lass','landlady','sistren','ladies','wenches','sorority','bellgirl','duchess',
    'ballerina','chicks','fiancee','fillies','wives','suitress','paternity','she',
    'businesswoman','masseuses','heroine','doe','busgirls','girlfriends','queens',
    'sisters','mistresses','stepmother','grooms','daughter','minxes','cowgirl',
    'lady','daughters','mezzo','saleswoman','mistress','hostess','nuns','maids',"mrs.",
    'headmistresses','lasses','congresswoman','airwoman','housewife','priestess','barwomen',
    'barnoesses','abbesses','handywoman','toque','sororities','stewardesses',
    'filly','czarina','stepdaughters','herself','girls','lionesses','lady','vagina','hers','masseuse','cows','aunts',
    'wench','toques','wife','lioness','sorceress','effeminate','mother','lesbians',
    'female','waitresses','skene_gland','stepdaughter','womb','businesswomen',
    'heiress','waitress','headmistress','woman','governess','godess','bride','grandma','bride','gal','lesbian','ladies','girl',
    'grandmother','mare','maternity','hens','nuns','maidservants','seamstress','busgirl','heroines'
]


DEFAULT_FEMALE_SPECIAL = [
    'dowry',
    'womb','pregnant','pregnancy','breast','abortion','miscarriage','clitoris',
    'menopause','vagina','uterus','ovaries'
]

In [45]:
dataDir = './data/wikitext-2'

In [46]:
corpus = data.Corpus(dataDir)
ntokens = len(corpus.dictionary)

In [47]:
occupations = "carpenter mechanician construction worker laborer driver sheriff mover developer \
farmer guard chief janitor lawyer cook physician ceo analyst manager supervisor salesperson \
editor designers accountant auditor writer baker clerk cashier counselors attendant teacher \
sewer librarian assistant cleaner housekeeper nurse receptionist hairdressers secretary"
occupations = occupations.split()
occupations = [o for o in occupations if o in corpus.dictionary.word2idx]
len(occupations)

28

In [48]:
def sortbybias(d):
    d_s = sorted(d.items(), key = lambda t: t[1] )
    return d_s

In [55]:
def gender_ratios_m_f(output_data_dir,file, model):
    n = 0
    tot = 0 
    print("Gender Ratios...")
    with open(file,'r') as f:
        data = json.load(f)
    bias_record = {}
    for words in data:
        #if (data[words]['m']+data[words]['f']!=0 and data[words]['f']!=0 and data[words]['m']!=0):
        if (data[words]['m']+data[words]['f'] > 5):
            score = data[words]['m']/(data[words]['m']+data[words]['f'])
            if words in occupations:
                tot+=score
                n +=1
            rec = {"b_score" : score}
            data[words].update(rec)
            bias_record[words] = json.dumps(data[words])
    #print(bias_record)
    #print(sortbybias(bias_record))
    output_file = os.path.join(output_data_dir, 'biased_words_m_f' + model)   
    print("Bias_score: ", (tot/n))
    #print('occupations used: ',n)
    with open(output_file,'w') as fp:
        json.dump(bias_record,fp, sort_keys=True) 
    print('n: ',n)
    return tot/n

In [50]:
def gender_ratios(output_data_dir,file, model):
    print("Gender Ratios...")
    with open(file,'r') as f:
        data = json.load(f)
    bias_record = {}
    for words in data:
        if (data[words]['m']+data[words]['f']!=0):
            score = data[words]['m']/(data[words]['m']+data[words]['f'])
            rec = {"b_score" : score}
            data[words].update(rec)
            bias_record[words] = json.dumps(data[words])
    #print(bias_record)
    #print(sortbybias(bias_record))
    output_file = os.path.join(output_data_dir, 'biased_words_' + model)    
    with open(output_file,'w') as fp:
        json.dump(bias_record,fp, sort_keys=True) 

In [51]:
def preservation_ratios_m_f(output_data_dir,file,model):
    n = 0
    tot = 0 
    print("Preservation Ratios...")
    with open(file,'r') as f:
        data = json.load(f)
    preserve_record = {}
    #yusu edited the next line
    for words in data:
        if (data[words]['m']+data[words]['f']!=0 ):
            if words in DEFAULT_MALE_SPECIAL or words in DEFAULT_FEMALE_SPECIAL:        
                #print(data[words])
                #yusu edited the next line
                if words in DEFAULT_MALE_SPECIAL:
                    score2 = data[words]['m']/(data[words]['m']+data[words]['f'])
                elif words in DEFAULT_FEMALE_SPECIAL:
                    score2 = data[words]['f']/(data[words]['m']+data[words]['f'])
                tot+=score2
                n +=1
                rec = {"p_score" : score2}
                data[words].update(rec)
                preserve_record[words] = json.dumps(data[words])
            
    # print(bias_record)
    # print(sortbybias(bias_record))
    output_file = os.path.join(output_data_dir, 'preserved_words_m_f'+ model)
    if n > 0:
        print("Preservation_score: ", (tot/n))
    else: 
        print('Special Words dont co occur with gender.')
    with open(output_file,'w') as fp:
        json.dump(preserve_record,fp, sort_keys=True)  


In [52]:
def get_cooccurrences(file, data, window):           
       
    with open(file, 'r') as fp:
        #print(fp)
        sentences = fp.read()

    male_nouns = DEFAULT_MALE_NOUNS
    female_nouns = DEFAULT_FEMALE_NOUNS
    n_grams = ngrams(sentences.split(), window)
    
    for grams in n_grams:
        pos = 1
        m = 0 
        f = 0 
        for w in grams:
                pos+=1
                if w not in data:
                    data[w]= {"m":0, "f":0}
                
                if pos==int((window+1)/2):
                    if w in male_nouns:
                        m = 1
                    if w in female_nouns:
                        f = 1
                    if m > 0:
                        for t in grams:
                            if t not in data:
                                data[t]= {"m":0, "f":0}
                            data[t]['m']+=1
                    if f > 0:
                        for t in grams:
                            if t not in data:
                                data[t]= {"m":0, "f":0}
                            data[t]['f']+=1
    return data

In [58]:
window=30
output_data_dir = './Bias/'
txtDir = './generated/'
models = os.listdir(txtDir)
biasList = []
models

['lambda_0.5', 'lambda_1', 'lambda_0', 'lambda_2', 'lambda_5']

In [56]:
for m in models:
    data ={}
    print("Processing: ",m)
    print("Files:",len(os.listdir(txtDir + m)))
    for file in os.listdir(txtDir + m):
        txt_path = txtDir + m + '/' + file
        data = get_cooccurrences(txt_path, data, window)
        #print(len(data))
    output_file = output_data_dir + m + '.json'
    with open(output_file,'w') as fp:
        json.dump(data,fp)
    bias = gender_ratios_m_f(output_data_dir,output_file, m)
    biasList.append(bias)

Processing:  lambda_0.5
Files: 1032
Gender Ratios...
Bias_score:  0.5036676286676287
n:  6
Processing:  lambda_1
Files: 1000
Gender Ratios...
Bias_score:  0.5491750488074018
n:  8
Processing:  lambda_0
Files: 1000
Gender Ratios...
Bias_score:  0.6993872793872794
n:  5
Processing:  lambda_2
Files: 1000
Gender Ratios...
Bias_score:  0.6052057845536106
n:  6
Processing:  lambda_5
Files: 1000
Gender Ratios...
Bias_score:  0.4737103174603175
n:  8
